In [1]:
import pandas as pd
from binance.client import Client
import plotly.graph_objects as go
from hyperopt import fmin, hp, tpe
import numpy as np

api_key = "A6X3Bl8R7rZweC5eSOT0It3JXrDZwbDmVhmSEty6BEjXKTOk1kQIbC605jqvkoj2"
api_secret = "ovF6YXVbZsICLLx91jFeBX6kUknHa6j7K3J2ADxE1438WGEre9o5Ca1fIoX03ngI"

client = Client(api_key, api_secret)

In [2]:
# Definir o símbolo do par de moedas desejado
symbol = 'BTCUSDT'

# interval = '1h'
# Definir o intervalo de tempo desejado (30 minutos neste caso)
interval = Client.KLINE_INTERVAL_30MINUTE

limit = 24 * 180

klines = client.futures_klines(symbol=symbol, interval=interval, limit=limit)

data = []
for kline in klines:
    timestamp = pd.to_datetime(kline[0], unit='ms')
    open_price = float(kline[1])
    high_price = float(kline[2])
    low_price = float(kline[3])
    close_price = float(kline[4])
    volume = float(kline[5])

    data.append([timestamp, open_price, high_price, low_price, close_price, volume])

BinanceAPIException: APIError(code=-1130): Data sent for parameter 'limit' is not valid.

In [ ]:
df = pd.DataFrame(data, columns=['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
df

,Timestamp,Open,High,Low,Close,Volume
0,2023-11-26 14:30:00,37358.7,37444.7,37268.5,37328.1,7671.362
1,2023-11-26 15:00:00,37328.2,37444.2,37279.3,37435.0,5638.920
2,2023-11-26 15:30:00,37435.0,37496.0,37384.0,37418.0,3949.330
3,2023-11-26 16:00:00,37418.0,37492.7,37212.0,37284.5,8248.569
4,2023-11-26 16:30:00,37284.5,37333.3,37126.0,37212.1,12600.058
...,...,...,...,...,...,...
715,2023-12-11 12:00:00,42346.6,42398.3,42262.2,42270.1,4302.311
716,2023-12-11 12:30:00,42270.1,42324.9,42184.2,42218.0,3766.829
717,2023-12-11 13:00:00,42218.0,42267.5,42106.6,42141.9,3783.050
718,2023-12-11 13:30:00,42141.8,42185.3,41900.4,42020.0,13893.349


In [ ]:
# Taxa de opearacao
taxa_operacao = 0.001

# Capital Inicial
capital = 250
quantidade_moeda = 0

quantidade_moeda_compra = 0.001
quantidade_moeda_venda = 0.001

num_std = 3.15
window = 15

df['MA'] = df['Close'].rolling(window= window).mean()
df['STD'] = df['Close'].rolling(window= window).std()

df['Upper'] = df['MA'] + num_std * df['STD']
df['Lower'] = df['MA'] - num_std * df['STD']

In [ ]:
#Construção de alvos
periodos = 1

#Alvo - Retorno
df["Retorno"] = df["Close"].pct_change(periodos)
df["Alvo"] = df["Retorno"].shift(periods=-periodos)

In [ ]:
df["Regra"] = np.where(df["Close"] > df["Upper"], 1, 0)
df["Regra"] = np.where(df["Close"] < df["Lower"], -1, df["Regra"])

In [ ]:
df["Tp_Operaocao"] = np.where(df["Close"] > df["Upper"], "Venda", "Holding")
df["Tp_Operaocao"] = np.where(df["Close"] < df["Lower"], "Compra", df["Tp_Operaocao"])

df["Trade"] = df["Regra"] * df["Alvo"]
df["Retorno_Trader_BB"] = df["Trade"].cumsum()

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = df['Timestamp'],
        y = df['Retorno_Trader_BB'],
        line = {'color': 'green'},
        name = 'Close'
    )
)

ratio = 1.77
height = 660
width = ratio * height

# Adicionar marcações de compra e venda
compra_indices = df.loc[df["Tp_Operaocao"].isin(["Compra"]), ['Retorno_Trader_BB', 'Timestamp']]
venda_indices = df.loc[df["Tp_Operaocao"].isin(["Venda"]), ['Retorno_Trader_BB', 'Timestamp']]

fig.add_trace(go.Scatter(x=compra_indices['Timestamp'], y=compra_indices['Retorno_Trader_BB'], mode='markers', name='Compra', marker=dict(color='orange', size=5)))
fig.add_trace(go.Scatter(x=venda_indices['Timestamp'], y=venda_indices['Retorno_Trader_BB'], mode='markers', name='Venda', marker=dict(color='red', size=5)))

fig.update_layout(
    height = height,
    width = width,
    template = 'plotly_white',
    title = f'{symbol} Banda de Bollinger'
)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Candlestick(
        x = df['Timestamp'],
        open= df['Open'],
        close=df['Close'],
        high=df['High'],
        low=df['Low'],
        name = 'Candlestick'
    )
)

fig.add_trace(
    go.Scatter(
        x = df['Timestamp'],
        y = df['Close'],
        line = {'color': 'blue'},
        name = 'Close'
    )
)

fig.add_trace(
    go.Scatter(
        x = df['Timestamp'],
        y = df['MA'],
        line = {'color': 'blue'},
        name = 'media_movel'
    )
)

fig.add_trace(
    go.Scatter(
        x = df['Timestamp'],
        y = df['Upper'],
        name='banda_superior'
    )
)

fig.add_trace(
    go.Scatter(
        x = df['Timestamp'],
        y = df['Lower'],
        name= 'banda_inferior'
    )
)

ratio = 1.77
height = 660
width = ratio * height

# Adicionar marcações de compra e venda
compra_indices = df.loc[df["Tp_Operaocao"].isin(["Compra"]), ['Close', 'Timestamp']]
venda_indices = df.loc[df["Tp_Operaocao"].isin(["Venda"]), ['Close', 'Timestamp']]

fig.add_trace(go.Scatter(x=compra_indices['Timestamp'], y=compra_indices['Close'], mode='markers', name='Compra', marker=dict(color='green', size=5)))
fig.add_trace(go.Scatter(x=venda_indices['Timestamp'], y=venda_indices['Close'], mode='markers', name='Venda', marker=dict(color='red', size=5)))

fig.update_layout(
    height = height,
    width = width,
    template = 'plotly_white',
    title = f'{symbol} Banda de Bollinger'
)

In [ ]:
def calcular_lucro_bollinger(df, media_movel, num_std):
    """
    """
    df['MA'] = df['Close'].rolling(window= int(media_movel)).mean()
    df['STD'] = df['Close'].rolling(window= int(media_movel)).std()
    df['Upper'] = df['MA'] + num_std * df['STD']
    df['Lower'] = df['MA'] - num_std * df['STD']

    capital = 250
    capital_op = capital
    quantidade_moeda = 0

    operacoes_dict = {
        'capital': [],
        'qtde_moeda': [],
        'qtde_moeda_operacao': [],
        'valor': [],
        'tp_operacao': [],
        'lucro': [],
        'date': []
    }

    operacoes_df = pd.DataFrame(data= operacoes_dict)

    for index, row in df.iterrows():

        quantidade_moeda_compra = 0.001
        quantidade_moeda_venda = 0.002

        # Comprar a moeda
        if row['Close'] < row['Lower'] and capital >= row['Close'] * quantidade_moeda_compra * (1.001) and quantidade_moeda < 0.002:

            # Capital antes operacao
            capital_inicial = capital

            # Valor compra USD
            valor_compra = row['Close'] * quantidade_moeda_compra

            # Taxa Operacao USDT
            taxa_operacao = valor_compra * 0.001

            # Compra moeda carteira
            quantidade_moeda += quantidade_moeda_compra

            # Valor final
            valor_final = valor_compra * 1.001

            capital -= valor_final

            #Lucro
            lucro = capital - capital_inicial

            operacoes_df.loc[len(operacoes_df)] =   {
                                                        'capital': capital, 
                                                        'qtde_moeda': quantidade_moeda, 
                                                        'qtde_moeda_operacao' : quantidade_moeda_compra, 
                                                        'valor': valor_final,
                                                        'tp_operacao': 'Compra',
                                                        'lucro': lucro,
                                                        'date': row['Timestamp']
                                                    }
        
        # Vender a moeda
        elif row['Close'] > row['Upper'] and quantidade_moeda >= quantidade_moeda_venda:

            # Capital antes operacao
            capital_inicial = capital

            # Retirada moeda carteira
            quantidade_moeda -= quantidade_moeda

            # Valor venda USD
            valor_venda = row['Close'] * quantidade_moeda_venda

            # Taxa Operacao USDT
            taxa_operacao = valor_venda * 0.001
            
            valor_final = valor_venda - taxa_operacao

            capital += valor_final
            lucro = capital - capital_inicial

            operacoes_df.loc[len(operacoes_df)] =   {
                                                        'capital': capital, 
                                                        'qtde_moeda': quantidade_moeda, 
                                                        'qtde_moeda_operacao' : quantidade_moeda_venda, 
                                                        'valor': valor_final,
                                                        'tp_operacao': 'Venda',
                                                        'lucro': lucro,
                                                        'date': row['Timestamp']
                                                    }

    return operacoes_df